## To do 
* Optimize LightGBM 
* Feature engineering 
* different shift and rolling timeframe
* different shift and rolling measures (mean, median, std)  --remove rolling/shift
* Resample! 

### Done 

1. recursive CSV

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

import lightgbm as lgb
from catboost import CatBoostRegressor

from optuna.samplers import TPESampler

from sklearn.model_selection import StratifiedKFold, GroupKFold, TimeSeriesSplit, KFold, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# SCALER_NAME = "None" 
# SCALER = MinMaxScaler() 

EPOCHS = 10000     
EARLY_STOPPING = 30

FOLDS = 5

#predict only monday data
PRED_MONDAY = False

LAG_FEATURES = True

sns.set(font_scale = 1)

OPTUNA = True
TASK = "GPU"  #GPU

In [ ]:
train_original = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv", index_col = 0)
test_original = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv", index_col = 0)
submission = pd.read_csv("../input/tabular-playground-series-mar-2022/sample_submission.csv", index_col = 0)

In [ ]:
def basic_feats(df):
    df= df.copy(deep = True)
    
    df["time"] = pd.to_datetime(df["time"])
    df["dir_x_y"] = df['x'].astype('str') + df['y'].astype('str')+ df['direction']
    #df.drop(["x","y","direction"],axis =1 , inplace = True)
    
    df["minute"] = df["time"].dt.minute
    df["hour"] = df["time"].dt.hour    
    df["day"] = df["time"].dt.day
    df["month"] = df["time"].dt.month
    df["dayofweek"]= df["time"].dt.weekday
    
    #New features
    df["x+y"] = df["x"]+df["y"]
    df['x_y'] = df['x'].astype('str') + df['y'].astype('str')
    df['hour_direction'] = df['hour'].astype('str') + df['direction'].astype('str')
    
    #df['afternoon'] = (df['time'].dt.hour > 12).astype('int')
    #df['dayofyear'] = df['time'].dt.dayofyear
    #df['inverse_dayofyear'] = 365 - df['time'].dt.dayofyear
    
    return df 

train = basic_feats(train_original)
test = basic_feats(test_original)

In [ ]:
if PRED_MONDAY:
    #only predict monday
    print("Predicting Monday only")
    train[train["dayofweek"]==0].reset_index(drop= True)

In [ ]:
train

Scaling 

In [ ]:
num_col = []
for col in train.columns:
    if train[col].dtypes != "object" and col != "congestion" and col !="time":
        num_col.append(col)
        
scaler = StandardScaler()
train[num_col] = scaler.fit_transform(train[num_col])
test[num_col] = scaler.transform(test[num_col])

## Rolling & Lag Features 

In [ ]:
shift_list = [1 
              ,2,3,4, 8
             ]
roll_window = [4,8, 12]

In [ ]:
def shift_vals(df, shift_list, groupby):
    df = df.copy(deep = True)
    
    for i in shift_list:
        #df[f"shift_{i}"]  = df.groupby(groupby)['congestion'].shift(1, fill_value=0)
        df[f"shift_{i}"] = df.groupby(groupby)['congestion'].transform(lambda s: s.shift(i, fill_value=0))
    return df

train = shift_vals(train,shift_list,groupby= 'dir_x_y')

In [ ]:
def rolling(df, roll_window, groupby):
    df = df.copy(deep = True)
    
    for i in roll_window:
        df[f"rolling_mean_{i}"] = df.groupby(groupby)['shift_1'].transform(lambda s: s.rolling(i, min_periods=1).mean())
        df[f"rolling_median_{i}"] = df.groupby(groupby)['shift_1'].transform(lambda s: s.rolling(i, min_periods=1).median())
        df[f"rolling_std_{i}"] = df.groupby(groupby)['shift_1'].transform(lambda s: s.rolling(i, min_periods=1).std())
    return df.fillna(0)

train = rolling(train,roll_window, groupby= 'dir_x_y')

In [ ]:
#Check 
train[train["dir_x_y"]=="00EB"].head(10)

# Encoding 
### Assumption 
There is a relationship in the direction of the highways \
i.e. EB highway becomes NB at some point therefore NB is affected by EB's congestion 

I cant seem to find a way to identify this relationship at this time, I therefore will apply **2 types of encodings** on the data: 
* LabelEncoder - to identify any sort of relationship (albet a poor one) 
* Onehotencoder - categorical engineering 

In [ ]:
# label encoder
# encoder = LabelEncoder()
# train["dir_x_y_LE"] = encoder.fit_transform(train["dir_x_y"])
# test ["dir_x_y_LE"] = encoder.transform(test["dir_x_y"])

# #Onehot
# # have to concatenate as test does not include all the feats as train 
# all_df = pd.concat([train.assign(ds=1),test.assign(ds=0)],axis =0)
# all_df = pd.get_dummies(all_df)
# test = all_df[all_df["ds"]==0].drop(["congestion","ds"],axis =1)
# train = all_df[all_df["ds"]==1].drop(["ds"],axis =1)

In [ ]:
str_list = [] 
num_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)
        
for col in str_list:
    encoder = LabelEncoder()
    encoder.fit(train[col])
    train[col] = encoder.transform(train[col])

    for label in np.unique(test[col]):
        if label not in encoder.classes_: 
            encoder.classes_ = np.append(encoder.classes_, label) 
    test[col] = encoder.transform(test[col])

# Split 

In [ ]:
X = train.drop(["congestion","time"],axis =1 )
y = train["congestion"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

features = train.drop(["time","congestion"],axis=1).columns 
print(len(features))
print([col for col in features])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = train.columns[(train.dtypes != 'object') & (train.columns != 'congestion') & (train.columns != 'time')]
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Optuna 

In [ ]:
import optuna 
def objective(trial):
    params = {
        "random_state":trial.suggest_categorical("random_state", [2022]),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.0001, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators": 1000,
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'task_type': trial.suggest_categorical('task_type', [TASK]),
        'loss_function': trial.suggest_categorical('loss_function', ['MAE']),
        'eval_metric': trial.suggest_categorical('eval_metric', ['MAE'])
    }

    model = CatBoostRegressor(**params)
    X_train_tmp, X_valid_tmp, y_train_tmp, y_valid_tmp = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    model.fit(
        X_train_tmp, y_train_tmp,
        eval_set=[(X_valid_tmp, y_valid_tmp)],
        early_stopping_rounds=35, verbose=0
    )
        
    y_train_pred = model.predict(X_train_tmp)
    y_valid_pred = model.predict(X_valid_tmp)
    train_mae = mae(y_train_tmp, y_train_pred)
    valid_mae = mae(y_valid_tmp, y_valid_pred)
    
    print(f'MAE of Train: {train_mae}')
    print(f'MAE of Validation: {valid_mae}')
    
    return valid_mae

In [ ]:
TRIALS = 100
TIMEOUT = 3600


if OPTUNA:
    sampler = TPESampler(seed=42)

    study = optuna.create_study(
        study_name = 'cat_parameter_opt',
        direction = 'minimize',
        sampler = sampler,
    )
    study.optimize(objective, n_trials=TRIALS)
    print("Best Score:",study.best_value)
    print("Best trial",study.best_trial.params)
    
    best_params = study.best_params
    
    X_train_tmp, X_valid_tmp, y_train_tmp, y_valid_tmp = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    model_tmp = CatBoostRegressor(**best_params, n_estimators=30000, verbose=1000).fit(X_train_tmp, y_train_tmp, eval_set=[(X_valid_tmp, y_valid_tmp)], early_stopping_rounds=35)

# Model

In [ ]:
  
def fit_model(X_train,y_train,X_test,y_test,test_df=None):
    
    if OPTUNA:
        model = CatBoostRegressor(**best_params, n_estimators=model_tmp.get_best_iteration(), verbose=1000).fit(X_train, y_train)
    else:
        model = CatBoostRegressor(
            verbose=1000,
            early_stopping_rounds=10,
            random_seed=2022,
            max_depth=12,
            task_type='GPU',
            learning_rate=0.035,
            iterations=30000,
            loss_function='MAE',
            eval_metric= 'MAE'
        ).fit(X_train, y_train)
    
    train_preds =model.predict(X_test) 
    
    #predict test data
    if test_df is None:
        test_preds = []
    else:
        test_preds = model.predict(test_df)
    
    mae = mean_absolute_error(y_test, train_preds)
    print("\nMAE:", mae)
    print("r2: ",r2_score(y_test, train_preds)) 
    
    return train_preds, test_preds, mae, model

train_preds, test_preds, mae ,model= fit_model(X_train, y_train, X_test, y_test, None)

## CV 

In [ ]:
kfold = KFold(n_splits= FOLDS,shuffle = False)

In [ ]:
def cross_val(X,y, test):
    
    test_predictions = []
    lgb_scores = []

    for idx, (train_idx, val_idx) in enumerate(kfold.split(X, y)):

        print("\n",10*"=", f"Fold={idx+1}", 10*"=")

        X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx,]
        X_valid, y_val = X.iloc[val_idx,:], y.iloc[val_idx,]
        
        train_preds, test_preds, mae ,model= fit_model(X_train[features], y_train, X_valid[features], y_val, test)                          
                                                       
        lgb_scores.append(mae)
        test_predictions.append(test_preds)
        
        del X_valid
        del y_val

    print("Mean Validation MAE :", np.mean(lgb_scores))
    return test_predictions

#test_predictions = cross_val(X,y, test= None)

In [ ]:
# submission["congestion"] = np.round(np.mean(test_predictions, axis =0))
# submission.to_csv("submission_ml.csv")

# plt.figure(figsize = (20,8))
# sns.lineplot(data = train.iloc[-8000:], x = train.iloc[-8000:].index.date,  y = "congestion", label = "actual")
# sns.lineplot(x = test.index.date,  y = submission["congestion"], label = "predicted")
# plt.show()

# Recursive 

In [ ]:
sub = pd.read_csv("../input/tabular-playground-series-mar-2022/sample_submission.csv",index_col=0)

FREQUENCY = 20 #prediction interval in minutes

start_date = min(test["time"]) 
end_date = max(test["time"])
print(start_date)
print(end_date)

In [ ]:
def multi_step_recursive(start_date, end_date, freq, sub, train_i, test_i):
    delta = pd.DateOffset(minutes = freq)

    all_df = pd.concat([train_i.assign(ds="a"),test_i.assign(ds="b")],axis =0)
    
    #Shift and rolling features
    if LAG_FEATURES:
        all_df = shift_vals(all_df,shift_list, groupby= "dir_x_y_LE")
        all_df = rolling(all_df,roll_window, groupby= "dir_x_y_LE")
        
    while start_date <= end_date:
        
        print("\n############ Start date" , start_date, " ############")
        #Select slice to predict
        test_split = all_df [  (all_df["time"]>= start_date ) & (all_df["time"]< start_date+delta) ][features]

        X = all_df[ all_df["time"]< start_date][features]
        y = all_df[ all_df["time"]< start_date]["congestion"]
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        
        #predict 1 timeframe - full data
        #one_period_preds, model = fit_model_full(X, y, test_df =test_split)
        one_period_preds = cross_val(X,y,test_split)
        
        #Add predicted test data back to all_df
        test_split["congestion"] = np.mean(one_period_preds, axis =0)
        all_df.loc[test_split.index, "congestion"]  = test_split["congestion"]
        print(np.mean(one_period_preds, axis =0))
        
        #update submission 
        sub.loc[test_split.index , "congestion"] =  np.mean(one_period_preds, axis =0)
        
        if LAG_FEATURES:
            all_df = shift_vals(all_df,shift_list, groupby= "dir_x_y_LE")
            all_df = rolling(all_df,roll_window, groupby= "dir_x_y_LE")
        
        #update start date
        start_date += delta
    
    return sub

In [ ]:
#sub_recursive  = multi_step_recursive(start_date, end_date, FREQUENCY, sub.copy(deep=True), train, test)

In [ ]:
# sub_recursive["congestion"] = np.round(sub_recursive["congestion"] )
# sub_recursive.to_csv("sub_recursive.csv")
# sub_recursive

In [ ]:
# plt.figure(figsize = (20,8))
# sns.lineplot(data = train.iloc[-8000:], x = train.iloc[-8000:]["time"],  y = "congestion", label = "actual")
# sns.lineplot(x = test["time"],  y = sub_recursive["congestion"], label = "predicted")
# plt.show()